In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2, keras, pickle
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

## Cargamos el modelo y el las labels

In [2]:
model = keras.models.load_model("data/model/model_train.h5")
infile = open('data/data8.pickle','rb')
new_dict = pickle.load(infile)
labels = new_dict["labels"]
window_name = 'Detect traffic signs'
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:
def is_this_sign(sign, ids_sign):
    return True in [sign == i for i in ids_sign]

In [4]:
def put_name_sign(image, sign, location=(50, 50), fontScale=0.5, color=(0, 0, 255), font=cv2.FONT_HERSHEY_SIMPLEX, thickness=1):
    width, heigh = cv2.getTextSize(sign, cv2.FONT_HERSHEY_SIMPLEX, fontScale, thickness)[0]
    location = (location[0]-width, location[1]+heigh+3)
    image = cv2.putText(image, sign, location, font, fontScale, color, thickness, cv2.LINE_AA) 

In [5]:

def capture_sign(image, x, y , w, h):
    sign_arr = np.array([image[y:y+h,x:x+w,:]])
    if sign_arr.shape[0] > 0 and sign_arr.shape[1] > 0:
        sign_scale = np.array([cv2.resize(sign_arr[0], (32,32))])[:,:,:,0:1]
        datagen.fit(sign_scale)
        return np.argmax(model.predict(sign_scale))
    return None

In [6]:
def detect_sign(imag=[], cascade="stop_cascade.xml", ids_sign = [14], color_box=(0, 0, 255), color_text=(0, 0, 255), thickness=2):
    URL = 'data/traffic_cascades/'
    sign_cascade = cv2.CascadeClassifier(f'{URL}{cascade}')
    imag1 = imag.copy()
    rectangles = sign_cascade.detectMultiScale(imag1, 1.05, 3)
    for (x,y,w,h) in rectangles:
        sign = capture_sign(imag1,x,y,w,h)
        print(sign, labels[sign])
        if is_this_sign(sign, ids_sign):
            cv2.rectangle(imag1, (x,y), (x+w, y+h), color_box, thickness)
            put_name_sign(imag1, labels[sign], location=(x+w//2, y+h), color=color_text)
    return imag1

In [7]:
def run_video_capture():
    capt = cv2.VideoCapture(0)
    while True:
        res,video = capt.read(0)
        video = detect_sign(video)
        video = detect_sign(video, "circular_cascade.xml", [1,2,3,4,5,6,8,9], (255,0,0), (255,0,0))
        video = detect_sign(video, "triangular_cascade.xml", [12,14,19,20,21,22,23,24,25,26,27,28,29,30,31,32], (0,255,0), (0,255,0))
        cv2.imshow(window_name, video)
        key = cv2.waitKey(1)
        if key == 27:
            break
    capt.release()
    cv2.destroyAllWindows() 

In [8]:
def main():
    run_video_capture()

In [9]:
main()

11 Right-of-way at the next intersection
13 Yield
13 Yield
18 General caution
15 No vehicles
18 General caution
41 End of no passing
12 Priority road
1 Speed limit (30km/h)
18 General caution
1 Speed limit (30km/h)
13 Yield
13 Yield
11 Right-of-way at the next intersection
2 Speed limit (50km/h)
18 General caution
12 Priority road
38 Keep right
13 Yield
26 Traffic signals
18 General caution
4 Speed limit (70km/h)
13 Yield
4 Speed limit (70km/h)
15 No vehicles
28 Children crossing
28 Children crossing
11 Right-of-way at the next intersection
15 No vehicles
4 Speed limit (70km/h)
26 Traffic signals
1 Speed limit (30km/h)
18 General caution
13 Yield
4 Speed limit (70km/h)
15 No vehicles
25 Road work
28 Children crossing
13 Yield
13 Yield
13 Yield
13 Yield
18 General caution
15 No vehicles
13 Yield
26 Traffic signals
11 Right-of-way at the next intersection
28 Children crossing
28 Children crossing
1 Speed limit (30km/h)
15 No vehicles
13 Yield
28 Children crossing
13 Yield
13 Yield
13 Yie

28 Children crossing
28 Children crossing
15 No vehicles
18 General caution
18 General caution
18 General caution
38 Keep right
13 Yield
18 General caution
13 Yield
11 Right-of-way at the next intersection
2 Speed limit (50km/h)
10 No passing for vehicles over 3.5 metric tons
13 Yield
18 General caution
12 Priority road
13 Yield
13 Yield
13 Yield
18 General caution
22 Bumpy road
28 Children crossing
15 No vehicles
11 Right-of-way at the next intersection
18 General caution
13 Yield
12 Priority road
28 Children crossing
18 General caution
18 General caution
11 Right-of-way at the next intersection
11 Right-of-way at the next intersection
11 Right-of-way at the next intersection
13 Yield
1 Speed limit (30km/h)
26 Traffic signals
13 Yield
13 Yield
22 Bumpy road
13 Yield
15 No vehicles
13 Yield
13 Yield
34 Turn left ahead
13 Yield
1 Speed limit (30km/h)
18 General caution
18 General caution
15 No vehicles
12 Priority road
41 End of no passing
12 Priority road
13 Yield
4 Speed limit (70km/h